In [12]:
from API_call import API_call
from gamePks import *
import statsapi as mlb
import pandas as pd

from tables import *
import relationships

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()

In [2]:
db.db_engine.table_names()

['actions',
 'credits',
 'game_player_links',
 'game_team_links',
 'games',
 'hit_data',
 'matchups',
 'pitch_data',
 'pitches',
 'players',
 'plays',
 'runners',
 'team_records',
 'teams',
 'venues']

In [47]:

    (SELECT
        id
    FROM 
        teams
    WHERE
        name = 'New York Yankees')

In [48]:
db.db_engine.execute(stmt).fetchall()

[(147,)]

In [109]:
# Yankees games on my birthday
stmt= """
SELECT 
    pk
FROM
    games
WHERE
    pk 
IN
    (
    SELECT
        gamePk
    FROM
        game_team_links
    WHERE
        team_id 
    IN
        (SELECT
            id
        FROM 
            teams
        WHERE
            name = 'New York Yankees')
    )
AND
    strftime('%m-%d',dateTime)='06-09'
    """

In [110]:
pk = db.db_engine.execute(stmt).first()[0]

In [111]:
call = API_call(pk)

In [112]:
# item for sublist in list for item in sublist

game_batting= list(
    { k for keys in call.game_player_links 
     for k in keys if k.startswith('stats_batting') }
)

In [113]:
season_batting = list(
    { k for keys in call.game_player_links 
     for k in keys if k.startswith('seasonStats_batting') }
)

In [114]:
game_batting

['stats_batting_doubles',
 'stats_batting_rbi',
 'stats_batting_caughtStealing',
 'stats_batting_runs',
 'stats_batting_gamesPlayed',
 'stats_batting_sacBunts',
 'stats_batting_catchersInterference',
 'stats_batting_totalBases',
 'stats_batting_triples',
 'stats_batting_note',
 'stats_batting_groundIntoTriplePlay',
 'stats_batting_stolenBasePercentage',
 'stats_batting_atBatsPerHomeRun',
 'stats_batting_homeRuns',
 'stats_batting_pickoffs',
 'stats_batting_flyOuts',
 'stats_batting_baseOnBalls',
 'stats_batting_intentionalWalks',
 'stats_batting_strikeOuts',
 'stats_batting_hits',
 'stats_batting_hitByPitch',
 'stats_batting_leftOnBase',
 'stats_batting_groundOuts',
 'stats_batting_sacFlies',
 'stats_batting_stolenBases',
 'stats_batting_plateAppearances',
 'stats_batting_groundIntoDoublePlay',
 'stats_batting_atBats']

In [116]:
season_batting

['seasonStats_batting_groundIntoDoublePlay',
 'seasonStats_batting_triples',
 'seasonStats_batting_rbi',
 'seasonStats_batting_gamesPlayed',
 'seasonStats_batting_ops',
 'seasonStats_batting_avg',
 'seasonStats_batting_baseOnBalls',
 'seasonStats_batting_groundIntoTriplePlay',
 'seasonStats_batting_hitByPitch',
 'seasonStats_batting_sacBunts',
 'seasonStats_batting_hits',
 'seasonStats_batting_slg',
 'seasonStats_batting_atBats',
 'seasonStats_batting_runs',
 'seasonStats_batting_babip',
 'seasonStats_batting_stolenBases',
 'seasonStats_batting_obp',
 'seasonStats_batting_groundOuts',
 'seasonStats_batting_stolenBasePercentage',
 'seasonStats_batting_homeRuns',
 'seasonStats_batting_totalBases',
 'seasonStats_batting_strikeOuts',
 'seasonStats_batting_flyOuts',
 'seasonStats_batting_catchersInterference',
 'seasonStats_batting_pickoffs',
 'seasonStats_batting_sacFlies',
 'seasonStats_batting_leftOnBase',
 'seasonStats_batting_caughtStealing',
 'seasonStats_batting_doubles',
 'seasonSta

In [11]:
pk = gamePks['2018'][606]

ins = db.insert_game(pk,replace=True)

pd.DataFrame.from_records(ins)

,game,table,insert_status,records_count,records_added,reason
0,530000,games,success,1,1,None
1,530000,plays,success,65,65,None
2,530000,pitches,success,270,270,None
3,530000,pitch_data,success,264,264,None
4,530000,hit_data,success,41,41,None
5,530000,actions,success,12,12,None
6,530000,runners,success,73,73,None
7,530000,credits,success,81,78,None
8,530000,matchups,success,65,65,None
9,530000,venues,success,1,0,None


In [8]:
for table in tables:
    db.db_engine.execute(f'delete from {table} where gamePk={pk}')

In [9]:
for table in tables:
    r = db.db_engine.execute(f"select count(*) from {table} where gamePk={pk}")
    print(r.fetchone()[0])

0
0
0
0
0
0
0
0
0
0
0


In [10]:
r.fetchone()[0]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
pks = gamePks['2019'][500:550]

calls = [API_call(pk) for pk in pks]
#item for sublist in list for item in sublist
records = [link for call in calls for link in call.game_player_links]

In [ ]:
df = pd.DataFrame.from_records(records)

In [ ]:
df['stats_pitching_gamesPitched'].value_counts()

In [ ]:
df['gameStatus_isCurrentBatter'].value_counts()

In [ ]:
# #TEST: which columns, if any, don't have any values
# dfs = {}
# for table in db.db_engine.table_names():
#     df = pd.read_sql(f"select * from {table}",db.db_engine)
#     dfs[table]=df


# for k in dfs.keys():
#     df = dfs[k]
#     for col in df.columns:
#         if df[col].isna().sum()==df.shape[0]:
#             print(k,col)

# pks = [x[0] for x in session.query(Game.pk).all()]

# calls = [API_call(pk) for pk in pks]

# dfs_fromApi = {}
# for k in calls[0].__dict__.keys():
#     if k[0] != '_':
#         records = [x for call in calls for x in call.__dict__[k]]
#         df = pd.DataFrame.from_records(records)
#         dfs_fromApi[k] = df

In [ ]:
call = API_call(567055)

In [ ]:
liveData = call._result['liveData']

In [ ]:
box_score = liveData['boxscore']

In [ ]:
away = box_score['teams']['away']

In [ ]:
players = away['players']

In [ ]:
player_dicts = []
for k in players.keys():
    player_dicts.append(flatten_dicts(players[k]))



In [ ]:
df = pd.DataFrame.from_records(player_dicts)

In [ ]:
df.info()

In [ ]:
df['status_description'].value_counts()

In [ ]:
df.info(verbose=True)

In [ ]:
stmt = """
select 
    pk,
    count(pk) as count
from
    games
group by 
    pk
having 
    count > 1
"""

In [ ]:
db.db_engine.execute(stmt).fetchall()